In [3]:
import numpy as np
import math

# Float to Spike converter
## FP Input Neuron

### Version 1

In [98]:
alpha = 17


def fp_d2s(x, K = 8, elim = 4, w = 1, alpha = 20):


    man_start = alpha
    exponent = 0

    while man_start > x and exponent < 2**elim - 1:
        exponent += 1
        man_start = man_start / 2


    print("man_start:\t", man_start)   
    print("exponent:\t", exponent)   
    print("exp elim:\t", 2**elim )

    T = man_start * 2* 2**(-K) * np.array([float(2 ** (K - i)) for i in range(1, K + 1)]).astype(np.float32)

    print("T: \t\t", T)
    print("x:\t\t", x)
    print("α:\t\t", alpha)

    exponent = format(abs(exponent), "b")

    while len(exponent) < elim:
        exponent = "0" + exponent
    
    # mantessa
    z = []
    for t in T:
        if t <= x:
            x = x - t
            z.append("1")   
        else:
            z.append("0")


    man = ''.join(z)

    return exponent + man


print("output:\t\t", fp_d2s(x =  5, K = 4, elim = 4))

man_start:	 5.0
exponent:	 2
exp elim:	 16
T: 		 [5.    2.5   1.25  0.625]
x:		 5
α:		 20
output:		 00101000


### version 2

In [4]:
alpha = 17


def fp_d2s(x, K = 8, elim = 4, w = 1, alpha = 20):
    t = 0 #current time step
    vM = np.zeros(K) #voltage of mantessa output
    vE = np.zeros(elim) #voltage of exponent output
    z = np.zeros(K+elim)

    man_start = alpha * 2** - min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha)))))

    exponent = min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha)))))

    m = man_start * 2* 2**(-K) * np.array([float(2 ** (K - i)) for i in range(1, K + 1)]).astype(np.float32)
    e = 16 * 2**(-elim) * np.array([float(2 ** (elim - i)) for i in range(1, elim + 1)]).astype(np.float32)

    vE[0] = exponent
    vM[0] = x

    # TODO  combine different voltages into 1 i.e. [e.0.0.0.m.0.0.0.0.0]...
    # ... rather than have them seperate
    while t < K + elim:
        #for calculating exponent
        if t < elim:
            if vE[t] >= e[t]:
                z[t] = 1
                vE[t] = vE[t] - e[t]  

            # copy over value once reduced.
            if t + 1 < elim:
                vE[t + 1] =  vE[t] # no need to reduce further as this has already been shortened.

        #for calculating mantessa
        else:
            if vM[t - elim] >= m[t - elim]:
                z[t] = 1
                vM[t - elim] = vM[t - elim] - m[t - elim]  

            # copy over value once reduced.
            if t + 1 < K + elim:
                vM[t - elim + 1] =  vM[t - elim] # no need to reduce further as this has already been shortened.
    
        t += 1
    
    return z

print("output:\t\t", str(fp_d2s(x =  5, K = 6, elim = 4, alpha = 20)))


[0. 0. 0. 0. 0. 0.] [2. 2. 0. 0.]
output:		 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0.]


### Version 3   


In [68]:
alpha = 20

def fp_d2s(x, K = 8, elim = 4, w = 1, alpha = 20):
    # current timestep
    t = 0 
    # membrane voltage
    v = np.zeros(K + elim) 
    # output spike train
    z = np.zeros(K + elim)

    # starting value for the mantessa scale
    man_start = alpha * 2** - min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha)))))
    print(man_start)
    # the integer exponent value
    exponent = min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha)))))
    # mantessa scale, starting at mantessa start
    m = man_start * 2* 2**(-K) * np.array([float(2 ** (K - i)) for i in range(1, K + 1)]).astype(np.float32)
    # exponent scale, dependend on length of elim
    e = 16 * 2**(-elim) * np.array([float(2 ** (elim - i)) for i in range(1, elim + 1)]).astype(np.float32)

    # placing the values of exponent and mantessa into output train
    v[0] = exponent
    v[elim] = x


    while t < K + elim:
        #for calculating exponent
        if t < elim:
            if v[t] >= e[t]:
                z[t] = 1
                v[t] = v[t] - e[t]  

            # copy over value once reduced.
            if t + 1 < elim:
                v[t + 1] =  v[t] # no need to reduce further as this has already been shortened.

        #for calculating mantessa
        else:
            if v[t] >= m[t - elim]:
                z[t] = 1
                v[t] = v[t] - m[t - elim]  

            # copy over value once reduced.
            if t + 1 < K + elim:
                v[t + 1] =  v[t] # no need to reduce further as this has already been shortened.
    
        t += 1

    
    return z

print("output:\t\t", str(fp_d2s(x =  4, K = 4, elim = 4, alpha = 20)))


2.5
output:		 [0. 0. 1. 1. 1. 1. 0. 0.]


In [110]:
def narray_to_string(array):
    string = ""
    for i in range(len(array)):
        string = string + str(int(array[i]))
    return string

narray_to_string(fp_d2s(x =  5, K = 6, elim = 4, alpha = 20))

'0010100000'

# Dev 101
### creating a single step (no loop) exponent value and starting value (for scalar pipeline)

In [71]:
x = 2.3
alpha = 20
exponent = 0
elim = 4
man_start = alpha

while man_start > x and exponent < 2**elim - 1:
    exponent += 1
    man_start = man_start / 2

print("origianl start:\t", man_start)
  
print("exponent actual:", exponent) 

# min (exponent limit, ceil of log2 of the number of divisions that can fit in alpha)
print("\ntest 1:\t\t", min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha))))))

# alpha * (factor of multiplicaton i.e. x = 7, start should be 5, therefore 20(alpha) * 0.25...
# ... which is achieved with 2 ^ - exponent)

print("test 2:\t\t", alpha * 2** - min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha))))))

origianl start:	 1.25
exponent actual: 4

test 1:		 4
test 2:		 1.25


# Spike to Decimal Converter

### version 2

In [111]:
def fp_s2d(x, K = 8, elim = 4, w = 1, alpha = 20):
    exp = int(x[:elim], 2)
    man = list(x[elim:])

    man_start = alpha

    print("exp :", exp)
    print("man :", man)

    while exp > 0:
        exp -= 1
        man_start = man_start / 2
        
    print("man_start: ", man_start)   
    
    T = man_start * 2* 2**(-K) * np.array([float(2 ** (K - i)) for i in range(1, K + 1)]).astype(np.float32)

    print(T)
    # mantessa
    z = 0

    for i in range(len(man)):
        if man[i] == "1":
            z += T[i]

    return z

print(fp_s2d("0010100000", K = 6, elim = 4))

exp : 2
man : ['1', '0', '0', '0', '0', '0']
man_start:  5.0
[5.      2.5     1.25    0.625   0.3125  0.15625]
5.0


# Current Alternative

main drawback is it's limit on being unable to convert input values which are either 0 or alpha-1

In [ ]:
import bitarray.util as bu
import math
def encode(x, alpha, exp_bits, mantissa_bits):
    # Calculate maximum mantissa value
    max_exp = (2 ** exp_bits) - 1
    print(max_exp)
    # Calculate what integer equivalent of mantissa of one will be
    mantissa_one = 2 ** mantissa_bits  #TODO function POW(2'.. check
    print(mantissa_one)
    # Get fraction of alpha we need to encode
    relative_x = x / alpha
    print(relative_x)
    assert relative_x < 1.0
    print(relative_x)
    # Calculate integer exponent (will be < 1) and clamp
    if x != 0:
        exponent = math.ceil(math.log2(relative_x)) #TODO look into log2 and ceil
        print(exponent)
        exponent = max(-max_exp, min(0, exponent)) #TODO math
        print(exponent)
    #exponent = 0 if x == 0: else exponent = 19
    # Calculate value this exponent will represent
    exponent_value = 2 ** exponent
    print("exponent value:", exponent_value)


    # Calculate floating point mantissa value
    mantissa = relative_x / exponent_value
    print(mantissa)
    assert mantissa < 1.0
    # Convert mantissa to fixed point bitarray and (negated) exponent to bitarray
    return (bu.int2ba(round(mantissa * mantissa_one), length=mantissa_bits),
            bu.int2ba(-exponent, length=exp_bits))
            
def decode(value, alpha):
    # Calculate what integer equivalent of mantissa of one will be
    mantissa_one = 2.0 ** len(value[0])
    # Extract components
    exp = bu.ba2int(value[1])
    mantissa = bu.ba2int(value[0])
    # Re-assemble floating point value
    return alpha * (2.0 ** -exp) * (mantissa / mantissa_one)
alpha = 20.0
code = encode(19, alpha, 4, 4)
print(code)
print(decode(code, alpha))

# FS Conversion


In [6]:
# Few-Spikes Neuron
# x = input 
# K = timesteps K = 1, ... t 
# α = value representation range (cap)

def fs(x, K = 8, alpha = 20):
    t = 0
    fx = 0
    v =  np.zeros(K)
    z = np.zeros(K) 

    T = alpha * 2**(-K) * np.array([float(2 ** (K - i)) for i in range(1, K + 1)]).astype(np.float32)
    h = alpha * 2**(-K) * np.array([float(2 ** (K - i)) for i in range(1, K + 1)]).astype(np.float32)
    d = alpha * 2**(-K) * np.array([float(2 ** (K - i)) for i in range(1, K + 1)]).astype(np.float32)

    print(h)

    v[0] = x

    while t < K:         
        # spike if voltage > threshold, reset if spike.
        if v[t] >= T[t]:
            z[t] = 1
            v[t] = v[t] - h[t]  

        # copy over value once reduced.
        if t + 1 < K:
            v[t + 1] =  v[t] # no need to reduce further as this has already been shortened.

        # sum voltage if spike
        fx += z[t] * h[t] #* 0.5
        t += 1

    return z, fx # outputs tuple of spike train and sum (sum for configuration)

In [16]:
print(fs(17.0))

[10.        5.        2.5       1.25      0.625     0.3125    0.15625
  0.078125]
(array([1., 1., 0., 1., 1., 0., 0., 1.]), 16.953125)


### Current method for FS

In [70]:

fun = lambda pars, dt: pars[1] * 2**(-pars[0])

print(2**(-8))

print(fun((8.0, 20.0), 2))

0.00390625
0.078125


### Dev method for FP

In [65]:
"""
derived_params=[("mantissa_scale", create_dpf_class(lambda pars, dt: pars[1] * 2 ** -fminf((2** pars[2] - 1), fmaxf(0.0, ceil(log2(1 / ( input / pars[1]))))) * 2 * 2**(-pars[0]))),
                ("exponent_scale", create_dpf_class(lambda pars, dt: 16.0 * 2 ** ( - pars[2])))], #TODO pars[0] = K, pars[1] = Alpha

5.0
[5.    2.5   1.25  0.625] <- m
output:		 [0. 0. 1. 0. 1. 0. 1. 1.]

"""
K, alpha, elim, x = 8, 20, 4, 7

K = 4.0         #pars[0]
alpha = 20.0    #pars[1]
elim = 4.0      #pars[2]
input = 7       #pars[3]

print("x / alpha                                                                     :", x / alpha)
print("log (x / alpha)                                                               :", math.log2(1 /(x / alpha)))
print("math.ceil(math.log2(1 /(x / alpha))                                           :", math.ceil(math.log2(1 /(x / alpha))))
print("max(0, math.ceil(math.log2(1 /(x / alpha))))                                  :", max(0, math.ceil(math.log2(1 /(x / alpha)))))
print("2**elim - 1                                                                   :", 2**elim - 1)
print("min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha))))):             :", min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha))))))
print("alpha * 2**                                                                   :")
print("alpha * 2** - min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha))))):", alpha * 2** - min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha))))))

man_start = alpha * 2** - min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha)))))


#mantissa_scale = lambda pars, dt: pars[1] * 2**(-pars[0])
mantissa_scale = lambda pars, dt: pars[1] * 2** - min((2**pars[2] - 1), max(0, math.ceil(math.log2(1 /(pars[3] / pars[1])))))

print(mantissa_scale([8.0, 20.0, 4.0, 7.0], 2) * 128)

#print(man_start)

x / alpha                                                                     : 0.35
log (x / alpha)                                                               : 1.5145731728297582
math.ceil(math.log2(1 /(x / alpha))                                           : 2
max(0, math.ceil(math.log2(1 /(x / alpha))))                                  : 2
2**elim - 1                                                                   : 15.0
min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha))))):             : 2
alpha * 2**                                                                   :
alpha * 2** - min((2**elim - 1), max(0, math.ceil(math.log2(1 /(x / alpha))))): 5.0
640.0


In [73]:
(2**(-x / alpha))

6.276672783174006

In [79]:
10 * 2**(-8)

5.0